# Sinkhorn to GMSH and K3D plots

In [1]:
import numpy as np

## Disk

In [2]:
binsize = 128
vertices = np.load(f'../arrays/source_image_toeplitz_disk_{binsize}.npy')

In [3]:
vertices = np.hstack((vertices,np.zeros((vertices.shape[0], 1)))) / vertices.max() * 0.5

In [4]:
# meshes are enumerated from left bottom corner to top right corner
# first changes x, then y

# binsize=4

#  node_tags:
#  12 13 14 15
#  8   9 10 11
#  4   5  6  7
#  0   1  2  3

#  quad-node-tags:
#  0-1-5-4,   1-2-6-5,    2-3-7-6
#  4-5-9-8,   5-6-10-9,   6-7-11-10
#  8-9-13-12, 9-10-14-13, 10-11-15-14

quad_tags = np.array([[i + binsize * j,
                       i + binsize * j + 1,
                       i + binsize * j + binsize + 1,
                       i + binsize * j + binsize ] 
                      for j in range(binsize - 1)
                      for i in range(binsize - 1)])

faces_quad_for_k3d = np.array([[fournodes[i],
                                fournodes[(i+1) % 4],
                                fournodes[i]] 
                               for fournodes in quad_tags 
                               for i in range(4)])

In [5]:
import k3d

plot = k3d.plot()
plot += k3d.mesh(vertices.astype(np.float32), faces_quad_for_k3d.astype(np.uint32), wireframe=True, color=2)
plot.display()

Output()

In [6]:
plot.camera = [0.2, -0.4, 1,
              0, 0, 0,
              -0.5, 0.7, 0.4]

In [7]:
import pymesh

mesh = pymesh.meshio.form_mesh(vertices, quad_tags)
pymesh.meshio.save_mesh('sinkhornmesh.msh', mesh)

## Ball

In [8]:
binsize = 64
vertices = np.load(f'../arrays/source_image_toeplitz_ball_{binsize}.npy')
# vertices = np.load('../arrays/bins_ball_8.npy')
vertices /= vertices.max()
dim = 3
# meshes are enumerated from left bottom corner to top right corner
# first changes z, then x, then y

# binsize = 4

#  node_tags:
#  3(19,35,51)   7(23,39,55)  11(27,43,59)  15(31,47,63)
#  2(18,34,50)   6(22,38,54)  10(26,42,58)  14(30,46,62)
#  1(17,33,49)   5(21,37,53)   9(25,41,57)  13(29,45,61)
#  0(16,32,48)   4(20,36,52)   8(24,40,56)  12(28,44,60)

# quad-node-tags:
# ----------------------------
# all quad on hyperplanes z*y
# 0-1-17-16, 1-2-18-17, 2-3-19-18,| 4-5-21-20,...| 8... | 12-13-29-28, ..., 14-15-31-30, # 12
# 16-17-33-32, ... | 20-21-37-36, ...|24 ... | 28..., 30-31-47-46,
# 32-33-49-48, ... | 36 | 40 | 44 ..., 46-47-63-62

zy = np.array([[k + binsize * j + binsize ** 2 * i,
                k + binsize * j + binsize ** 2 * i + 1,
                k + binsize * j + binsize ** 2 * i + 1 + binsize ** 2,
                k + binsize * j + binsize ** 2 * i + binsize ** 2]
            for i in range(binsize - 1)
            for j in range(binsize)
            for k in range(binsize - 1)
            ])

# all quad on hyperplanes x*y
# 0-4-20-16, 1-5-21-17, 2-6-22-18, 3-7-23-19, | 4-8-24-20, 5-9-25-21,..., |8-12-28-24,...., 11-15-31-27,
# 16-20-36-32, ..., 19-23-39-35,| 20 ...,|24 ..., 27-31-47-43,
# 32-36-52-48, ..., | 36-40-56-52, ..., | 40-44-60-56, ..., 43-47-63-59

xy = np.array([[k + binsize * j + binsize ** 2 * i, 
                k + binsize * j + binsize ** 2 * i + 1 * binsize,
                k + binsize * j + binsize ** 2 * i + 1 * binsize + binsize ** 2,
                k + binsize * j + binsize ** 2 * i + binsize ** 2]
            for i in range(binsize - 1) 
            for j in range(binsize - 1)
            for k in range(binsize)
            ])

# all quads on hyperplanes z*x
# 0-1-5-4, 1-2-6-5, 2-3-7-6,| 4-5-9-8, ...| 8..., 10-11-15-14, 
# 16-17-21-20, ..., | 20...| 24, 26-27-31-30,
# 32...,| 36..., | 40..., 42-43-47-46,
# 48-49-53-52, ... | 52..., | 56..., 58-59-63-62


zx = np.array([[k + binsize * j + binsize ** 2 * i,
                k + binsize * j + binsize ** 2 * i + 1,
                k + binsize * j + binsize ** 2 * i + 1 + binsize,
                k + binsize * j + binsize ** 2 * i + binsize]
            for i in range(binsize)
            for j in range(binsize - 1)
            for k in range(binsize - 1)
            ])

quad_tags = np.vstack([xy, zy, zx])
quad_tags.shape

(762048, 4)

In [9]:
faces_quad_for_k3d = np.array([[fournodes[i],
                                fournodes[(i + 1) % 4],
                                fournodes[i]] 
                               for fournodes in quad_tags 
                               for i in range(4)])

In [10]:
# vertices, voxels, quads-fours
binsize ** 3, (binsize - 1) ** 3, ((((binsize - 1) * (binsize - 1)) * binsize) * 3)

(262144, 250047, 762048)

In [11]:
plot = k3d.plot()
plot += k3d.mesh(vertices.astype(np.float32),
                 faces_quad_for_k3d.astype(np.uint32)[::binsize*2],
                 wireframe=True, color=2, linewidth=0.1)
plot.display()

Output()

# GMSH.gui + sinkhorn

In [12]:
import gmsh

gmsh.initialize()
gmsh.open('sinkhornmesh.msh')
c1 = gmsh.model.occ.addCircle(0., 0., 0., 0.5)
cl = gmsh.model.occ.addCurveLoop([c1])
gmsh.model.occ.synchronize()
gmsh.fltk.run()
gmsh.finalize()

In [13]:
gmsh.initialize()

# ----------------------- CREATE SHAPE --------------------------
gmsh.model.add("Disk")

c1 = gmsh.model.occ.addCircle(0., 0., 0., 0.5)
cl = gmsh.model.occ.addCurveLoop([c1])
pl = gmsh.model.occ.addPlaneSurface([cl])

gmsh.model.occ.synchronize()

gmsh.option.setNumber("Mesh.RecombineAll", 1)
gmsh.model.mesh.setRecombine(2, pl)
gmsh.option.setNumber("Mesh.Algorithm", 6) # 2 6 7 8
gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 2)

# ----------------------- GENERATE ------------------------------

gmsh.model.mesh.generate(2)
gmsh.model.mesh.recombine()
gmsh.model.mesh.refine()
gmsh.model.mesh.recombine()
gmsh.model.mesh.refine()
gmsh.fltk.run()
gmsh.finalize()